In [35]:
import datetime as datetime
import pandas as pd
import math
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pyodbc
import requests

short_start = "2019-2-01"
med_start = "2019-1-01"
long_start = "2018-01-01"
short_end = "2019-3-01"
med_end = "2019-3-01"
long_end = "2019-3-01"

drop_list = ['XRP' , 'XLM' , 'BCH' , 'LTC' , 'ADA' , 'TRX' , 'EOS' , 'BSV']
long_drop = ['currency' , 'platform' , 'application']

tfs = ['short' , 'med' , 'long']

df_all = dict.fromkeys(tfs)
df_benchmarks = dict.fromkeys(tfs)
returns_all = dict.fromkeys(tfs)
returns_benchmarks = dict.fromkeys(tfs)
results = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
net_return = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
ann_factor = dict.fromkeys(tfs)
total_return = dict.fromkeys(tfs)
percent_sd = dict.fromkeys(tfs)
summary = dict.fromkeys(tfs)
corr = dict.fromkeys(tfs)
betas = dict.fromkeys(tfs)
beta_results = dict.fromkeys(tfs)
total_return_excel = dict.fromkeys(tfs)
sortino_df = dict.fromkeys(tfs)
sortino = dict.fromkeys(tfs)
average_return = dict.fromkeys(tfs)

data_bletchley = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\\bletchley_history.xlsx' , index_col= 0)
data_coins = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bletchley.xlsx' , index_col=0)
data_bletchley['ERC20'] = data_bletchley['ethereum']
data_bletchley = data_bletchley.drop('ethereum' , axis=1)
data_bletch = data_bletchley

In [36]:
data_finance = data_bletch.fillna(method='ffill')
data_finance = data_finance.dropna()
index_data = data_finance
data_coins = data_coins.fillna(method='ffill')
data_coins = data_coins.dropna()

index_data.index = (pd.to_datetime(index_data.index.values,unit='ms'))
index_data.index = index_data.index.round('d')
df = index_data.join(data_coins, how='inner')

returns = df.pct_change()
returns = returns.dropna()

In [37]:
#filter master dataframes based on date ranges
returns_all['short'] = returns.loc[short_start:short_end]
returns_benchmarks['short'] = returns.loc[short_start:short_end]
df_all['short'] = df.loc[short_start:short_end]
df_benchmarks['short'] = df.loc[short_start:short_end]

returns_all['med'] = returns.loc[med_start:med_end]
returns_benchmarks['med'] = returns.loc[med_start:med_end]
df_all['med'] = df.loc[med_start:med_end]
df_benchmarks['med'] = df.loc[med_start:med_end]

returns_all['long'] = returns.loc[long_start:long_end]
returns_benchmarks['long'] = returns.loc[long_start:long_end]
df_all['long'] = df.loc[long_start:long_end]
df_benchmarks['long'] = df.loc[long_start:long_end]

#drop coins that aren't benchmarks
for c in drop_list:
    for t in tfs:
        returns_benchmarks[t] = returns_benchmarks[t].drop([c] , axis=1)
        df_benchmarks[t] = df_benchmarks[t].drop([c] , axis=1)

In [38]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-22,-0.016823,-0.032790,-0.016717,-0.017925,-0.014601,-0.027421,-0.023335,-0.021164,0.006033,-0.009196
2019-02-23,0.016702,0.033819,0.020317,0.017967,0.015669,0.025631,0.023803,0.026836,0.030705,0.027582
2019-02-24,0.045893,0.050313,0.040195,0.046050,0.040948,0.067348,0.016888,0.013235,-0.045528,0.021735
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.004059,0.030882


In [39]:
#drop sectors that don't have data back to 1/1/2018
for i in long_drop:
    df_benchmarks['long'] = df_benchmarks['long'].drop([i] , axis=1)
    returns_benchmarks['long'] = returns_benchmarks['long'].drop([i] , axis=1)

In [40]:
#calculate sortino to btc
for t in tfs:
    sortino_df[t] = pd.DataFrame()
    sortino[t] = pd.DataFrame(index = ['sortino'])
    for col in returns_benchmarks[t]:
        sortino_df[t][col] = returns_benchmarks[t][col] - returns_benchmarks[t]['BTC']
        avg_ret = np.mean(sortino_df[t][col])
        a = [r ** 2 for r in sortino_df[t][col] if r < 0]
        a_sum = np.sum(a)
        b = len(sortino_df[t][col])
        down_risk = np.sqrt(a_sum/b)
        if down_risk > 0.000001:
            sortino[t][col] = (avg_ret / down_risk)*math.sqrt(365)
        else:
            sortino[t][col] = 0

In [41]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-22,-0.016823,-0.032790,-0.016717,-0.017925,-0.014601,-0.027421,-0.023335,-0.021164,0.006033,-0.009196
2019-02-23,0.016702,0.033819,0.020317,0.017967,0.015669,0.025631,0.023803,0.026836,0.030705,0.027582
2019-02-24,0.045893,0.050313,0.040195,0.046050,0.040948,0.067348,0.016888,0.013235,-0.045528,0.021735
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.004059,0.030882


In [42]:
data_coins_week = data_coins.loc[short_start:short_end]
#data_coins_week = data_coins_week[:]
net_week_return = data_coins_week.pct_change(len(data_coins_week)-1)
ann_week_factor = 365.000/(len(data_coins_week))
data_coins_week_returns = pd.DataFrame()
data_coins_week_returns = data_coins_week.pct_change()
data_coins_week_returns = data_coins_week_returns.dropna()
total_week_return = net_week_return.tail(1)*100
data_coins_week = data_coins_week.append(total_week_return)
data_coins_week.index = data_coins_week.index[:-1].append(pd.Index(['return']))

In [43]:
data_coins_week

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
2019-02-01 00:00:00,3434.159070,106.940900,0.31008,0.081510,31.582380,113.855540,0.02515,2.320780,0.038370,63.784020
2019-02-02 00:00:00,3504.075450,107.600930,0.30883,0.081560,32.921310,115.791230,0.02635,2.375950,0.038890,64.382450
2019-02-03 00:00:00,3462.765390,109.470430,0.30530,0.081340,34.056510,119.396750,0.02557,2.391120,0.038550,64.364290
2019-02-04 00:00:00,3438.310100,106.533190,0.29945,0.079410,33.163810,117.557760,0.02517,2.358450,0.037850,63.267440
2019-02-05 00:00:00,3429.255370,106.857480,0.29733,0.078050,33.872030,117.918550,0.02670,2.385360,0.037690,62.616330
2019-02-06 00:00:00,3434.783920,106.494340,0.29381,0.075640,33.681330,115.625790,0.02578,2.344850,0.037340,60.059600
2019-02-07 00:00:00,3387.866700,104.071050,0.28696,0.073410,32.492230,113.693800,0.02527,2.330850,0.036370,62.074720
2019-02-08 00:00:00,3478.950920,106.282050,0.30471,0.077920,34.602420,117.799570,0.02730,2.454950,0.038490,65.547730
2019-02-09 00:00:00,3635.724300,118.499660,0.30931,0.080520,42.681750,128.417160,0.02650,2.698510,0.040350,66.661070
2019-02-10 00:00:00,3699.248260,119.028000,0.31124,0.080520,44.650460,127.399090,0.02619,2.794000,0.041060,65.832840


In [44]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.002041,-0.011593
2019-02-02,0.010041,0.011111,0.008623,0.010051,0.011235,0.004722,0.028551,0.030775,0.020359,0.006172
2019-02-03,0.015933,0.004272,0.006784,0.014922,0.014239,0.014904,0.012689,0.005754,-0.011789,0.017374
2019-02-04,-0.020945,-0.015801,-0.003981,-0.020153,-0.018528,-0.023694,-0.009049,-0.013480,-0.007062,-0.026831
2019-02-05,-0.003668,0.000808,-0.013358,-0.003647,-0.004557,-0.007102,0.018634,0.026514,-0.002633,0.003044
2019-02-06,0.001495,0.017993,-0.017855,0.002038,0.002691,-0.002001,0.016667,0.009467,0.001612,-0.003398
2019-02-07,-0.017216,-0.016160,-0.016814,-0.017138,-0.015711,-0.025014,0.001640,-0.006391,-0.013659,-0.022755
2019-02-08,-0.005118,0.004567,0.007774,-0.004129,-0.005254,0.001828,-0.008564,-0.007530,0.026885,0.021245
2019-02-09,0.099001,0.089244,0.074505,0.097697,0.093065,0.110015,0.069554,0.065363,0.045063,0.114955
2019-02-10,-0.001943,0.010255,0.007582,-0.000879,-0.002645,0.005028,0.015300,0.007550,0.017472,0.004459


In [45]:
print ('short start: ', returns_benchmarks['short'].index[0])
print ('medium start: ', returns_benchmarks['med'].index[0])
print ('long start: ', returns_benchmarks['long'].index[0])
print ('end: ', returns_benchmarks['short'].index[-2])
print ('end: ', returns_benchmarks['med'].index[-2])
print ('end: ', returns_benchmarks['long'].index[-2])

short start:  2019-02-01 00:00:00
medium start:  2019-01-01 00:00:00
long start:  2018-11-09 00:00:00
end:  2019-02-25 00:00:00
end:  2019-02-25 00:00:00
end:  2019-02-25 00:00:00


In [46]:
returns_benchmarks['short'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.002041,-0.011593
2019-02-02,0.010041,0.011111,0.008623,0.010051,0.011235,0.004722,0.028551,0.030775,0.020359,0.006172
2019-02-03,0.015933,0.004272,0.006784,0.014922,0.014239,0.014904,0.012689,0.005754,-0.011789,0.017374
2019-02-04,-0.020945,-0.015801,-0.003981,-0.020153,-0.018528,-0.023694,-0.009049,-0.013480,-0.007062,-0.026831
2019-02-05,-0.003668,0.000808,-0.013358,-0.003647,-0.004557,-0.007102,0.018634,0.026514,-0.002633,0.003044


In [47]:
print (data_coins_week.head(n=1))
data_coins_week.tail(n=1)

                            BTC       ETH      XRP      XLM       LTC  \
2019-02-01 00:00:00  3434.15907  106.9409  0.31008  0.08151  31.58238   

                           BCH      TRX      EOS      ADA       BSV  
2019-02-01 00:00:00  113.85554  0.02515  2.32078  0.03837  63.78402  


,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
return,11.784046,29.180108,5.64693,5.545332,43.889188,17.446081,-4.17495,53.009333,13.421944,6.079955


In [48]:
annualize = math.sqrt(365)
for t in tfs:
    results[t] = returns_benchmarks[t].describe()
    vol[t] = pd.DataFrame()
    vol[t] = results[t].loc['std']*annualize*100
    vol[t].name = 'annualized volatility'
    results[t] = results[t].append(df_benchmarks[t].iloc[0])
    results[t].index = results[t].index[:-1].append(pd.Index(['open price']))
    results[t] = results[t].append(df_benchmarks[t][-1:])
    results[t].index = results[t].index[:-1].append(pd.Index(['last price']))
    results[t] = results[t].append(vol[t])

In [49]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-22,-0.016823,-0.032790,-0.016717,-0.017925,-0.014601,-0.027421,-0.023335,-0.021164,0.006033,-0.009196
2019-02-23,0.016702,0.033819,0.020317,0.017967,0.015669,0.025631,0.023803,0.026836,0.030705,0.027582
2019-02-24,0.045893,0.050313,0.040195,0.046050,0.040948,0.067348,0.016888,0.013235,-0.045528,0.021735
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.004059,0.030882


In [50]:
for t in tfs:
    average_return[t] = pd.DataFrame()
    average_return[t] = returns_benchmarks[t]
    average_return[t].loc['mean'] = returns_benchmarks[t].mean(axis=0)
    returns_benchmarks[t] = returns_benchmarks[t][:-1]

In [51]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-22 00:00:00,-0.016823,-0.032790,-0.016717,-0.017925,-0.014601,-0.027421,-0.023335,-0.021164,0.006033,-0.009196
2019-02-23 00:00:00,0.016702,0.033819,0.020317,0.017967,0.015669,0.025631,0.023803,0.026836,0.030705,0.027582
2019-02-24 00:00:00,0.045893,0.050313,0.040195,0.046050,0.040948,0.067348,0.016888,0.013235,-0.045528,0.021735
2019-02-25 00:00:00,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26 00:00:00,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.004059,0.030882


In [52]:
for t in tfs:
    net_return[t] = df_benchmarks[t].pct_change(len(df_benchmarks[t])-1)
    ann_factor[t] = 365.000/(len(df_benchmarks[t]))
    total_return[t] = net_return[t].tail(1)*100
    total_return_excel[t] = net_return[t].tail(1)
    percent_sd[t] = results[t].iloc[2,:]*100
    results[t] = results[t].append(percent_sd[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['daily volatility']))
    #print total_return_excel[t]
    results[t] = results[t].append(total_return_excel[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['return']))
    #print results[t]
    #print average_return[t]/(percent_sd[t]*0.01).transpose()
    Sharpe_values = (average_return[t].loc['mean']/(percent_sd[t]*0.01))*annualize
    Sharpe_values = Sharpe_values.rename('sharpe_values')
    results[t] = results[t].append(Sharpe_values)
    results[t].index = results[t].index[:-1].append(pd.Index(['Sharpe']))
    results[t] = results[t].append(sortino[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['Sortino']))
#print (results[t])

In [53]:
returns_benchmarks['med'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-01 00:00:00,-0.046381,-0.050800,-0.056148,-0.046939,-0.042739,-0.057864,-0.040122,-0.046808,-0.012938,-0.038211
2019-01-02 00:00:00,0.046762,0.043521,0.042677,0.046447,0.043137,0.051576,0.036109,0.039667,0.047316,0.058320
2019-01-03 00:00:00,0.033371,0.035948,0.027412,0.033396,0.022256,0.053596,0.031989,0.027348,-0.019799,0.083028
2019-01-04 00:00:00,-0.041354,-0.043182,-0.034871,-0.041317,-0.036859,-0.054475,-0.028133,-0.029247,-0.000386,-0.027875
2019-01-05 00:00:00,0.018170,0.025959,0.024532,0.018870,0.013260,0.025122,0.048155,0.050412,0.002143,0.040172


In [54]:
for t in tfs:
    summary[t] = results[t].iloc[8:]
    corr[t] = returns_benchmarks[t].corr(method='spearman')
    betas[t] = returns_benchmarks[t].cov()
    beta_results[t] = pd.DataFrame()
    beta_results[t] = betas[t].iloc[0,:] / betas[t].iloc[3,0]
    corr[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_corr'+t+'.csv')
    summary[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_results'+t+'.csv')
beta_all = pd.DataFrame()
beta_all['MTD'] = beta_results['short']
beta_all['QTD'] = beta_results['med']
beta_all['YTD'] = beta_results['long']

In [55]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-22 00:00:00,-0.016823,-0.032790,-0.016717,-0.017925,-0.014601,-0.027421,-0.023335,-0.021164,0.006033,-0.009196
2019-02-23 00:00:00,0.016702,0.033819,0.020317,0.017967,0.015669,0.025631,0.023803,0.026836,0.030705,0.027582
2019-02-24 00:00:00,0.045893,0.050313,0.040195,0.046050,0.040948,0.067348,0.016888,0.013235,-0.045528,0.021735
2019-02-25 00:00:00,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26 00:00:00,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,0.004059,0.030882


In [56]:
summary['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,374.603942,313.478539,49.263316,156.644587,20.150812,13.096591,14.062167,43.673755,3434.159070,106.940900
last price,428.973158,383.730355,56.292252,180.175943,22.627771,16.104248,17.039814,50.158737,3838.841950,138.146370
annualized volatility,70.776515,65.037056,59.479418,69.695126,64.380846,86.189087,55.498549,55.067659,40.211013,87.771354
daily volatility,3.704612,3.404195,3.113295,3.648010,3.369848,4.511343,2.904927,2.882373,2.104741,4.594163
return,0.145138,0.224104,0.142681,0.150221,0.122921,0.229652,0.211749,0.148487,0.117840,0.291801
Sharpe,2.821506,3.971877,2.556234,2.897032,2.696021,3.394665,4.290120,2.732613,4.171401,4.507572
Sortino,0.746018,2.345549,-0.439462,0.812773,0.149126,2.505275,2.038163,-0.469103,0.000000,4.847868


In [57]:
summary['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,420.511599,337.862280,54.644247,175.290083,22.237102,15.347474,13.168780,39.924454,3773.237410,134.063560
last price,428.973158,383.730355,56.292252,180.175943,22.627771,16.104248,17.039814,50.158737,3838.841950,138.146370
annualized volatility,67.486192,74.492851,69.367589,67.549760,61.332376,85.291192,73.514160,76.024420,39.763406,85.399544
daily volatility,3.532389,3.899134,3.630866,3.535716,3.210283,4.464345,3.847907,3.979300,2.081312,4.470016
return,0.020122,0.135760,0.030159,0.027873,0.017568,0.049309,0.293955,0.256341,0.017387,0.030454
Sharpe,0.068669,1.028197,0.083875,0.136790,0.024344,0.337624,2.288715,1.921497,0.264752,0.354184
Sortino,-0.149695,1.496688,-0.114838,-0.032644,-0.257778,0.358729,3.852660,3.199121,0.000000,0.407788


In [58]:
summary['long']

,ten,twenty,forty,total,ERC20,BTC,ETH
open price,740.953740,637.900186,110.287135,311.303644,73.888661,6496.416980,216.812750
last price,428.973158,383.730355,56.292252,180.175943,50.158737,3838.841950,138.146370
annualized volatility,86.528023,97.925699,100.459342,87.303589,100.869080,64.653125,100.153329
daily volatility,4.529084,5.125665,5.258282,4.569679,5.279729,3.384099,5.242265
return,-0.421053,-0.398448,-0.489585,-0.421221,-0.321158,-0.409083,-0.362831
Sharpe,-1.677144,-1.240803,-1.727322,-1.655295,-0.776802,-2.393591,-1.008328
Sortino,0.279398,0.790403,-0.431769,0.294109,1.789240,0.000000,1.192032


In [59]:
corr['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.863077,0.869231,0.995385,0.990000,0.965385,0.763846,0.796154,0.273846,0.921538
twenty,0.863077,1.000000,0.769231,0.870769,0.857692,0.908462,0.883846,0.882308,0.327692,0.806154
forty,0.869231,0.769231,1.000000,0.876154,0.855385,0.900769,0.671538,0.683846,0.428462,0.892308
total,0.995385,0.870769,0.876154,1.000000,0.987692,0.970000,0.794615,0.824615,0.277692,0.922308
currency,0.990000,0.857692,0.855385,0.987692,1.000000,0.945385,0.774615,0.805385,0.324615,0.901538
platform,0.965385,0.908462,0.900769,0.970000,0.945385,1.000000,0.775385,0.800769,0.259231,0.927692
application,0.763846,0.883846,0.671538,0.794615,0.774615,0.775385,1.000000,0.988462,0.401538,0.677692
ERC20,0.796154,0.882308,0.683846,0.824615,0.805385,0.800769,0.988462,1.000000,0.370769,0.713846
BTC,0.273846,0.327692,0.428462,0.277692,0.324615,0.259231,0.401538,0.370769,1.000000,0.346154
ETH,0.921538,0.806154,0.892308,0.922308,0.901538,0.927692,0.677692,0.713846,0.346154,1.000000


In [60]:
corr['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.899180,0.907519,0.998018,0.994532,0.979768,0.816405,0.811620,0.296446,0.840807
twenty,0.899180,1.000000,0.853178,0.913397,0.895557,0.928571,0.934176,0.937047,0.322488,0.761586
forty,0.907519,0.853178,1.000000,0.915311,0.901162,0.928435,0.806152,0.788107,0.355161,0.798428
total,0.998018,0.913397,0.915311,1.000000,0.992344,0.983185,0.839576,0.835065,0.298838,0.842242
currency,0.994532,0.895557,0.901162,0.992344,1.000000,0.967738,0.815106,0.809433,0.314901,0.822761
platform,0.979768,0.928571,0.928435,0.983185,0.967738,1.000000,0.831921,0.825427,0.273821,0.847368
application,0.816405,0.934176,0.806152,0.839576,0.815106,0.831921,1.000000,0.988038,0.377990,0.701025
ERC20,0.811620,0.937047,0.788107,0.835065,0.809433,0.825427,0.988038,1.000000,0.345044,0.693233
BTC,0.296446,0.322488,0.355161,0.298838,0.314901,0.273821,0.377990,0.345044,1.000000,0.347232
ETH,0.840807,0.761586,0.798428,0.842242,0.822761,0.847368,0.701025,0.693233,0.347232,1.000000


In [61]:
corr['long']

,ten,twenty,forty,total,ERC20,BTC,ETH
ten,1.000000,0.932555,0.946011,0.998860,0.879409,0.662135,0.854508
twenty,0.932555,1.000000,0.926911,0.942934,0.954694,0.635743,0.838606
forty,0.946011,0.926911,1.000000,0.952970,0.889593,0.686479,0.853424
total,0.998860,0.942934,0.952970,1.000000,0.892512,0.661903,0.858160
ERC20,0.879409,0.954694,0.889593,0.892512,1.000000,0.656075,0.776406
BTC,0.662135,0.635743,0.686479,0.661903,0.656075,1.000000,0.572737
ETH,0.854508,0.838606,0.853424,0.858160,0.776406,0.572737,1.000000


In [62]:
beta_all

,MTD,QTD,YTD
ten,1.015908,0.999705,0.991451
twenty,0.865747,1.019796,1.078507
forty,0.802982,0.948902,1.101376
total,1.000000,1.000000,1.000000
currency,0.922208,0.903160,NaN
platform,1.220545,1.244787,NaN
application,0.687293,0.904670,NaN
ERC20,0.679528,0.913688,1.052525
BTC,0.219407,0.295134,0.598256
ETH,1.151127,1.121047,1.013297


In [63]:
coins_high_low = data_coins_week_returns.describe()
coins_high_low

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.004682,0.011314,0.002596,0.002742,0.016419,0.007435,-0.000975,0.018567,0.005481,0.002919
std,0.021281,0.046018,0.028984,0.034998,0.062407,0.045200,0.038818,0.055006,0.029985,0.034457
min,-0.045528,-0.128770,-0.056299,-0.066581,-0.089641,-0.127891,-0.104960,-0.120839,-0.054974,-0.053069
25%,-0.007062,-0.006851,-0.012309,-0.023728,-0.026212,-0.015402,-0.024360,-0.010606,-0.014316,-0.015861
50%,0.001679,0.008508,-0.002661,0.000000,0.022460,0.003069,-0.008839,0.017990,0.005216,-0.001695
75%,0.020359,0.024884,0.014903,0.027960,0.038736,0.023693,0.018241,0.034500,0.024965,0.016985
max,0.045063,0.114955,0.084053,0.087388,0.233490,0.126676,0.080332,0.179601,0.058290,0.109368


In [64]:
returns_benchmarks['short'].describe()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.005471,0.007077,0.004166,0.005532,0.004755,0.008016,0.006523,0.004123,0.004596,0.010839
std,0.037046,0.034042,0.031133,0.036480,0.033698,0.045113,0.029049,0.028824,0.021047,0.045942
min,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.045528,-0.128770
25%,-0.011583,-0.008879,-0.013358,-0.011642,-0.010306,-0.015214,-0.009049,-0.013480,-0.007062,-0.006851
50%,0.001495,0.004567,0.007582,0.002854,0.002648,0.004722,0.013522,0.005754,0.001679,0.008508
75%,0.017042,0.024830,0.020317,0.017967,0.015231,0.026317,0.018634,0.024836,0.017472,0.021735
max,0.099001,0.089244,0.074505,0.097697,0.093065,0.110015,0.069554,0.065363,0.045063,0.114955


In [65]:
data_coins_week.tail(n=1)

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
return,11.784046,29.180108,5.64693,5.545332,43.889188,17.446081,-4.17495,53.009333,13.421944,6.079955
